In [ ]:
!pip -q install tensorrt
!pip -q install pycuda

Dans ce Notebook, nous allons étudier comment utiliser un modèle optimisé avec TensorRT à l'aide de Cuda.



In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
import shutil

from tensorflow import keras
from matplotlib import pyplot as plt

<center><img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Olympidades2024/GestionCuda.jpg?raw=true" width=800></center>

### Chargement du moteur (engine)

On commence par créer une instance de logger qui va nous permettre d'afficher les informations pendant l'utilisation du moteur.

In [ ]:
import tensorrt as trt

# Construction de la class du logger
class MyLogger(trt.ILogger):
    def __init__(self):
        trt.ILogger.__init__(self)

    def log(self, severity, msg):
        print("%s : %s" %(severity,msg))
        pass

 Ensuite, on créé un ``runtime`` (environnement de travail) pour notre modèle puis on charge le moteur à l'aide de la fonction ``deserialize_cuda_engine`` que nous porpose ce runtime.

In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit

PRECISION = trt.int32

logger = MyLogger()
runtime = trt.Runtime(logger)
trt.init_libnvinfer_plugins(logger, namespace="")

with open("/content/model.engine", "rb") as f:
    engine = runtime.deserialize_cuda_engine(f.read())

 ### Création du contexte

Ensuite, il faut créer un contexte nécessaire pour lancer le modèle. Ce contexte va allouer une certaine quantité de mémoire afin de pouvoir travailler avec notre mmoteur.

In [ ]:
context = engine.create_execution_context()

### Allocation de l'espace mémoire hôte et GPU

Nous allons devoir allouer de l'espace en mémoire sur **l'hote** et sur le **GPU** pour réaliser des transferts de données entre eux.
Commençons par regarder combien de mémoire nous avons en GPU :

In [ ]:
cuda.mem_get_info()

Pour transférer des données entre l'hôte et le GPU, celles-ci doivent être transférées depuis un espace mémoire paginé vérrouillé de l'hôte. On va donc devoir réserver un espace mémoire sur l'hôte et le vérrouiller (cet espace ne pourra pas servir à autre chose et donc ne pourra pas être utlisé par un autre processus).

Les étapes à suivre pour utliser notre moteur sont les suivantes :
- Réserver un espace mémoire vérrouillé sur l'hote pour y placer l'entrée (**input_host_mem**) et la sortie du modèle (**output_host_mem**)
- Réserver un espace mémoire dans le GPU pour y placer ces mêmes informations (**input_device_mem** et **output_device_mem**)
- Placer les données d'entrées du modèle (notre image) dans l'espace mémoire hôte vérrouillé qu'on a pris soin de réserver au préalable (**input_host_mem**)
- Transférer le contenu de cet espace mémoire vers la mémoire du GPU (**input_device_mem**)
- Lancer les calculs avec le modèle TensorRT qu'on a défini ; les résultats seront écrits dans la mémoire du GPU (**output_device_mem**)
- Transférer les résultats de la mémoire GPU (**output_device_mem**) vers la mémoire vérrouillées de l'hote (**output_host_mem**)

On peut trouver des informations sur le format des tenseurs en entrée et en sortie de notre moteur :

In [ ]:
for binding in engine:
    print(binding)
    print(engine.get_tensor_shape(binding))
    print(trt.nptype(engine.get_tensor_dtype(binding)))

In [ ]:
engine.get_tensor_shape("input")

On va donc utiliser ces informations pour allouer les espaces en mémoire hôte et GPU :

In [ ]:
# Réservation de la mémoire pour l'entrée
size_input = trt.volume(engine.get_tensor_shape("input"))
input_host_mem = cuda.pagelocked_empty(size_input, trt.nptype(PRECISION))
input_device_mem = cuda.mem_alloc(input_host_mem.nbytes)

In [ ]:
# Réservation de la mémoire pour les sorties
output_device_mem = [];
format_sorties = [];
types_sorties = [];

size_output = trt.volume(engine.get_tensor_shape("output_0"))
output_host_mem = cuda.pagelocked_empty(size_output, trt.nptype(PRECISION))
output_device_mem.append(cuda.mem_alloc(output_host_mem.nbytes))
format_sorties.append(engine.get_tensor_shape("output_0"))
types_sorties.append(trt.nptype(engine.get_tensor_dtype("output_0")))

Il faut maintenant récupérer les adresses mémoires GPU. Ces adresses seront utiles au contexte de TensorRT précédemment ouvert pour pouvoir exécuter le modèle.

In [ ]:
# Récupère les adresses en GPU des buffers entrées / sorties
binding_entree = int(input_device_mem)

binding_sorties = []
for output_ in output_device_mem:
    binding_sorties.append(int(output_))

### Exécution d'une prédiction

On commence par récupérer une image du dataset :

In [ ]:
image = tf.keras.preprocessing.image.load_img("/content/Yoga.jpg",target_size=(192, 192))
image

Puis on convertit cette image en tenseur :

On copie maintenant ce tenseur dans l'espace mémoire d'entrée de l'hôte. Le tenseur est applati à l'aide de la fonction ``ravel()``:

In [ ]:
image = np.expand_dims(image,axis=0)                          # (1,192,192,3)
image = image.astype(np.int32)
np.copyto(input_host_mem,image.ravel())

In [ ]:
image

Transfert les données de l'image vers la mémoire GPU (transfert Host => Device) :

In [ ]:
cuda.memcpy_htod(input_device_mem, input_host_mem)

Exécution du modèle :

In [ ]:
bindings = [binding_entree, binding_sorties[0]]

In [ ]:
bindings

In [ ]:
context.execute_v2(bindings=bindings)

On transfert le résultat stocké en mémoire GPU vers la mémoire destination de l'hôte :

In [ ]:
output_device_mem

In [ ]:
output_host_mem = []
for i in range(len(output_device_mem)):
    output_host_mem.append(np.zeros(format_sorties[i],types_sorties[i]))

for i in range(len(output_host_mem)):
    cuda.memcpy_dtoh(output_host_mem[i], output_device_mem[i])
output_host_mem

## Visualisation du résultat

**Outputs**

A float32 tensor of shape [1, 1, 17, 3].

* The first two channels of the last dimension represents the yx coordinates (normalized to image frame, i.e. range in [0.0, 1.0]) of the 17 keypoints:
  * 0: nose,
  * 1, 2 : left eye, right eye,
  * 3, 4 : left ear, right ear
  * 5, 6 : left shoulder, right shoulder
  * 7, 8 : left elbow, right elbow
  * 9, 10 : left wrist, right wrist
  * 11, 12 : left hip, right hip
  * 13, 14 : left knee, right knee
  * 15, 16 : left ankle, right ankle

* The third channel of the last dimension represents the prediction confidence scores of each keypoint, also in the range [0.0, 1.0].

<center><img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Olympidades2024/keypoints.png?raw=true" width=500></center>

In [ ]:
# Par exemple, pour obtenir la coordonnée X de l'oeuil gauche :
output_host_mem[0][0][0][1][0]

In [ ]:
# Par exemple, pour obtenir la coordonnée Y de l'oeuil gauche :
output_host_mem[0][0][0][1][1]

In [ ]:
# Par exemple, pour obtenir la coordonnée X du nez :
output_host_mem[0][0][0][0][0]

In [ ]:
# Par exemple, pour obtenir la coordonnée Y du nez :
output_host_mem[0][0][0][0][1]

In [ ]:
output_host_mem[0][0][0][:][:]

In [ ]:
toto =output_host_mem[0][0][0][:][:]
toto[0]

In [ ]:
import cv2

def draw_points(img_cv2, coordonneesXY, line_width=1):
    width, height = (800,800)

    # Place les points
    for point in coordonneesXY:
      pt_Y = point[0]
      pt_X = point[1]

      # Calcule la position du point sur l'image
      pt_image_X = pt_X * width
      pt_image_Y = pt_Y * height

      # Affiche le point sur l'image
      img = cv2.circle(img_cv2, (int(pt_image_X),int(pt_image_Y)), radius=2, color=(0, 0, 255), thickness=-1)

    return img

image = cv2.imread("/content/Yoga.jpg")
image = cv2.resize(image,(int(800),int(800)),3)

draw_image = draw_points(image, output_host_mem[0][0][0][:][:])
draw_image

In [ ]:
import cv2

def draw_lines(img_cv2, coordonneesXY, line_width=1):
    width, height = (600,600)

    nez = [int(coordonneesXY[0][1]*width),int(coordonneesXY[0][0]*height)]
    oeuil_gauche = [int(coordonneesXY[1][1]*width),int(coordonneesXY[1][0]*height)]
    oeuil_droit = [int(coordonneesXY[2][1]*width),int(coordonneesXY[2][0]*height)]
    oreille_gauche = [int(coordonneesXY[3][1]*width),int(coordonneesXY[3][0]*height)]
    oreille_droite = [int(coordonneesXY[4][1]*width),int(coordonneesXY[4][0]*height)]
    epaule_gauche = [int(coordonneesXY[5][1]*width),int(coordonneesXY[5][0]*height)]
    epaule_droite = [int(coordonneesXY[6][1]*width),int(coordonneesXY[6][0]*height)]
    coude_gauche = [int(coordonneesXY[7][1]*width),int(coordonneesXY[7][0]*height)]
    coude_droite = [int(coordonneesXY[8][1]*width),int(coordonneesXY[8][0]*height)]
    poignet_gauche = [int(coordonneesXY[9][1]*width),int(coordonneesXY[9][0]*height)]
    poignet_droite = [int(coordonneesXY[10][1]*width),int(coordonneesXY[10][0]*height)]
    hanche_gauche = [int(coordonneesXY[11][1]*width),int(coordonneesXY[11][0]*height)]
    hanche_droite = [int(coordonneesXY[12][1]*width),int(coordonneesXY[12][0]*height)]
    genou_gauche = [int(coordonneesXY[13][1]*width),int(coordonneesXY[13][0]*height)]
    genou_droite = [int(coordonneesXY[14][1]*width),int(coordonneesXY[14][0]*height)]
    cheville_gauche = [int(coordonneesXY[15][1]*width),int(coordonneesXY[15][0]*height)]
    cheville_droite = [int(coordonneesXY[16][1]*width),int(coordonneesXY[16][0]*height)]


    centre_yeux = [((int(0.5*(coordonneesXY[1][1]*width+int(coordonneesXY[2][1]*width))))),((int(0.5*(coordonneesXY[1][0]*height+int(coordonneesXY[2][0]*height)))))]
    centre_epaules = [((int(0.5*(coordonneesXY[5][1]*width+int(coordonneesXY[6][1]*width))))),((int(0.5*(coordonneesXY[5][0]*height+int(coordonneesXY[6][0]*height)))))]
    centre_hanches =  [((int(0.5*(coordonneesXY[11][1]*width+int(coordonneesXY[12][1]*width))))),((int(0.5*(coordonneesXY[11][0]*height+int(coordonneesXY[12][0]*height)))))]


    # Lignes nez-centre des yeux
    img = cv2.line(img_cv2, (nez[0],nez[1]),(centre_yeux[0],centre_yeux[1]),color=(0, 0, 255), thickness=2)

    # Lignes centre des yeux - oueil gauche
    img = cv2.line(img_cv2, (oeuil_gauche[0],oeuil_gauche[1]),(centre_yeux[0],centre_yeux[1]),color=(0, 0, 255), thickness=2)

    # Lignes centre des yeux - oueil droit
    img = cv2.line(img_cv2, (oeuil_droit[0],oeuil_droit[1]),(centre_yeux[0],centre_yeux[1]),color=(0, 0, 255), thickness=2)

    # Lignes oeuil_gauche - oreille gauche
    img = cv2.line(img_cv2, (oeuil_gauche[0],oeuil_gauche[1]),(oreille_gauche[0],oreille_gauche[1]),color=(0, 0, 255), thickness=2)

    # Lignes oeuil_droit - oreille droite
    img = cv2.line(img_cv2, (oeuil_droit[0],oeuil_droit[1]),(oreille_droite[0],oreille_droite[1]),color=(0, 0, 255), thickness=2)

    # Lignes nez-centre_epaules
    img = cv2.line(img_cv2, (nez[0],nez[1]),(centre_epaules[0],centre_epaules[1]),color=(0, 0, 255), thickness=2)

    # Ligne centre_epaules - epaule_droit
    img = cv2.line(img_cv2, (epaule_droite[0],epaule_droite[1]),(centre_epaules[0],centre_epaules[1]),color=(0, 0, 255), thickness=2)

    # Ligne centre_epaules - epaule_gauche
    img = cv2.line(img_cv2, (epaule_gauche[0],epaule_gauche[1]),(centre_epaules[0],centre_epaules[1]),color=(0, 0, 255), thickness=2)

    # Ligne épaule_gauche - coude_gouche
    img = cv2.line(img_cv2, (epaule_gauche[0],epaule_gauche[1]),(coude_gauche[0],coude_gauche[1]),color=(0, 0, 255), thickness=2)

    # Ligne coude_gouche - poignet_gauche
    img = cv2.line(img_cv2, (poignet_gauche[0],poignet_gauche[1]),(coude_gauche[0],coude_gauche[1]),color=(0, 0, 255), thickness=2)

    # Ligne épaule_droite - coude_doit
    img = cv2.line(img_cv2, (epaule_droite[0],epaule_droite[1]),(coude_droite[0],coude_droite[1]),color=(0, 0, 255), thickness=2)

    # Ligne coude_droite - poignet_droite
    img = cv2.line(img_cv2, (poignet_droite[0],poignet_droite[1]),(coude_droite[0],coude_droite[1]),color=(0, 0, 255), thickness=2)

    # Ligne centre_epaules - centre_hanche
    img = cv2.line(img_cv2, (centre_hanches[0],centre_hanches[1]),(centre_epaules[0],centre_epaules[1]),color=(0, 0, 255), thickness=2)

    # Ligne centre_hanche - hanche_gauche
    img = cv2.line(img_cv2, (centre_hanches[0],centre_hanches[1]),(hanche_gauche[0],hanche_gauche[1]),color=(0, 0, 255), thickness=2)

    # Ligne centre_hanche - hanche_gauche
    img = cv2.line(img_cv2, (centre_hanches[0],centre_hanches[1]),(hanche_droite[0],hanche_droite[1]),color=(0, 0, 255), thickness=2)

    # Ligne hanche_gauche - genou_gauche
    img = cv2.line(img_cv2, (genou_gauche[0],genou_gauche[1]),(hanche_gauche[0],hanche_gauche[1]),color=(0, 0, 255), thickness=2)

    # Ligne hanche_droite - genou_droite
    img = cv2.line(img_cv2, (genou_droite[0],genou_droite[1]),(hanche_droite[0],hanche_droite[1]),color=(0, 0, 255), thickness=2)

    # Ligne genou_droite - cheville_droite
    img = cv2.line(img_cv2, (genou_droite[0],genou_droite[1]),(cheville_droite[0],cheville_droite[1]),color=(0, 0, 255), thickness=2)

    # Ligne genou_gauche - cheville_gauche
    img = cv2.line(img_cv2, (genou_gauche[0],genou_gauche[1]),(cheville_gauche[0],cheville_gauche[1]),color=(0, 0, 255), thickness=2)

    return img

image = cv2.imread("/content/Yoga.jpg")
image = cv2.resize(image,(int(600),int(600)),3)

draw_image = draw_lines(image, output_host_mem[0][0][0][:][:])
draw_image

Enfin pour terminer, on libère la mémoire allouée :

In [ ]:
input_device_mem.free()
for i in range(len(output_device_mem)):
    output_device_mem[i].free()
del input_host_mem
del output_host_mem
del context
del engine